In [16]:
import socket
import pandas as pd
import numpy as np
import subprocess
from cassandra.cluster import Cluster
from time import sleep
from sqlalchemy           import create_engine, text
from sqlalchemy.orm       import sessionmaker
from sqlalchemy           import URL
from pymongo.mongo_client import MongoClient
from pymongo.server_api   import ServerApi

### Conectando ao Cassandra

In [17]:
hostname = socket.gethostname()
ip_address = socket.gethostbyname(hostname)
port = 9042

cluster = Cluster([ip_address], port=port)
session = cluster.connect('cc6240')

### Funções

#### Create

In [18]:
def criar_tabelas():
    # instructor_teaches
    session.execute("CREATE TABLE instructor_teaches (id text, name text, dept_name text, salary float, course_id text, title text, semester text, primary key((course_id, id)));")
    session.execute("CREATE INDEX ON instructor_teaches(semester);")
    
    # course_department
    session.execute("CREATE TABLE course_department (dept_name text, building text, budget float, course_id text, title text, credits int, PRIMARY KEY (dept_name, course_id, title));")

    # course
    session.execute("CREATE TABLE course (course_id text, title text, dept_name text, credits int, PRIMARY KEY((course_id), dept_name));")
    
    # section
    session.execute("CREATE TABLE section (course_id text, sec_id text, semester text, year int, building text, room_number text, time_slot_id text, title text, PRIMARY KEY ((course_id), sec_id, semester, year));")
    session.execute("CREATE INDEX ON section(semester);")
    
    # students_course
    session.execute("CREATE TABLE students_course (dept_name text, building text, budget float, course_id text, title text, credits int, id text, name text, tot_cred  int, PRIMARY KEY ((name), title, dept_name));")
    session.execute("CREATE INDEX ON students_course(title);")

    # students_teaches
    session.execute("CREATE TABLE students_teaches (id_instructor text, name text, salary float, id_student text, name_student text, dept_name text, tot_cred int, PRIMARY KEY ((name),  name_student, dept_name));")
    session.execute("CREATE INDEX ON students_teaches(dept_name);")

    # student_advisor
    session.execute("CREATE TABLE student_advisor (id_instructor text, name_instructor text, id_student text, name_student text, PRIMARY KEY ((id_instructor, name_instructor, id_student, name_student)));")

    # classroom_section
    session.execute("CREATE TABLE classroom_section (building text, room_number text, capacity int, PRIMARY KEY ((building)));")

    # instructor_student
    session.execute("CREATE TABLE instructor_student (id_instructor text, name text, salary float, id_student text, name_student text, dept_name text, tot_cred int, PRIMARY KEY ((name), name_student, dept_name));")

#### Delete

In [19]:
def limpar_tabela(tabela):
    if tabela == 'student':
        students = session.execute('SELECT name FROM student;')
        for s in students:
            session.execute(f"DELETE FROM {tabela} WHERE name = '{s[0]}';")

    elif tabela == 'instructor':
        instructor = session.execute('SELECT name FROM instructor;')
        for i in instructor:
            session.execute(f"DELETE FROM {tabela} WHERE name = '{i[0]}';")
    
    elif tabela == 'course_department':
        course_department = session.execute('SELECT dept_name FROM course_department;')
        for i in course_department:
            session.execute(f"DELETE FROM {tabela} WHERE dept_name = '{i[0]}';")
    
    elif tabela == 'section':
        course_id = session.execute('SELECT course_id FROM section;')
        for i in course_id:
            session.execute(f"DELETE FROM {tabela} WHERE course_id = '{i[0]}';")
            
    elif tabela == 'students_course':
        name = session.execute('SELECT name FROM students_course;')
        for i in name:
            session.execute(f"DELETE FROM {tabela} WHERE name = '{i[0]}';")

    elif tabela == 'students_teaches':
        name = session.execute('SELECT name FROM students_teaches;')
        for i in name:
            session.execute(f"DELETE FROM {tabela} WHERE name = '{i[0]}';")

    elif tabela == 'instructor_teaches':
        name = session.execute('SELECT name FROM instructor_teaches;')
        for i in name:
            session.execute(f"DELETE FROM {tabela} WHERE name = '{i[0]}';")

def deletar_tabela(tabela):
    try:
        session.execute(f'DROP TABLE {tabela};')
    except:
        print(f'Tabela "{tabela}" não encontrada!')
        pass

#### Outras funções

In [20]:
def listar_colunas(tabela):
    instructor = session.execute(f'select * from {tabela};')
    for i in instructor:
        print(i)

def selecionar_instrutor(nomes=[]):
    prepared_statement = session.prepare("select * from instructor where name = ?;")
    for i in nomes:
        intructor = session.execute(prepared_statement, [i]).one()
        print(intructor)

def inserir_dados(name, dept_name, advisor, tot_cred):
    session.execute(f"INSERT INTO student (name, dept_name, advisor, tot_cred) VALUES ('{name}', '{dept_name}', '{advisor}', {tot_cred});")

### Conectando ao Postgres

In [21]:
url_object = URL.create(
    "postgresql+pg8000",
    username="dcxhxgql",
    password="bPlk_dl7Xc4l0WEaPCJIYR4dnO9kGZbz",
    host="motty.db.elephantsql.com",
    database="dcxhxgql",
)

engine = create_engine(url_object)
engine = create_engine(url_object)
Session = sessionmaker(bind=engine)
session_postgres = Session()
conn = engine.connect()

### Configurando as tabelas do Cassandra

In [22]:
deletar_tabela('course_department')
deletar_tabela('section')
deletar_tabela('students_course')
deletar_tabela('students_teaches')
deletar_tabela('course')
deletar_tabela('instructor_teaches')
deletar_tabela('student_advisor')
deletar_tabela('classroom_section')
deletar_tabela('instructor_student')

In [23]:
criar_tabelas()

#### Inserindo dados nas tabelas

In [24]:
limpar_tabela('instructor_teaches')

query = """
select
	i.id,
	i.name,
	i.dept_name,
	i.salary,
	c.course_id,
	c.title,
	t.semester
from
	instructor i
join
	teaches t on i.id = t.id
join
	course c on t.course_id = c.course_id;
"""
instructors = conn.execute(text(query)).all()

inst_teaches_cassandra = []
		
for inst in instructors:
    query = f"INSERT INTO instructor_teaches(id, name, dept_name, salary, course_id, title, semester) VALUES (%s, %s, %s, %s, %s, %s, %s);"
    session.execute(query, (inst[0], inst[1], inst[2], inst[3], inst[4], inst[5], inst[6]))
    inst_teaches_cassandra.append(inst)

session_postgres.close()

In [25]:
limpar_tabela('course_department')

query = """
SELECT 
    *
FROM
    department d 
JOIN
    course c on d.dept_name = c.dept_name;
"""
cursos = conn.execute(text(query)).all()

cursos_dept_cassandra = []
		
for curso in cursos:
    query = f"INSERT INTO course_department(dept_name, building, budget, course_id, title, credits) VALUES (%s, %s, %s, %s, %s, %s);"
    session.execute(query, (curso[0], curso[1], curso[2], curso[3], curso[4], curso[6]))
    cursos_dept_cassandra.append(curso)

session_postgres.close()

In [26]:
limpar_tabela('section')

query = """
SELECT 
    s.course_id,
    s.sec_id,
    s.semester,
    s.year,
    s.building,
    s.room_number,
    s.time_slot_id,
    c.title
FROM
    section s
JOIN
    course c ON s.course_id = c.course_id;
"""

sections = conn.execute(text(query)).all()

section_cassandra = []
	
for section in sections:
    query = f"INSERT INTO section(course_id, sec_id, semester, year, building, room_number, time_slot_id, title) VALUES (%s, %s, %s, %s, %s, %s, %s, %s);"
    session.execute(query, (section[0], section[1], section[2], section[3], section[4], section[5], section[6], section[7]))
    section_cassandra.append(section)

session_postgres.close()

In [27]:
limpar_tabela('students_course')

query = """
SELECT
    d.dept_name,
    d.building,
    d.budget,
    c.course_id,
    c.title,
    c.credits,
    s.id,
    s.name,
    s.tot_cred 
FROM
    department d  
JOIN
    course c on d.dept_name = c.dept_name 
JOIN
    student s on c.dept_name = s.dept_name;
"""
stud_course = conn.execute(text(query)).all()

stud_course_cassandra = []
		
for stud in stud_course:
    query = f"INSERT INTO students_course(dept_name, building, budget, course_id, title, credits, id, name, tot_cred) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);"
    session.execute(query, (stud[0], stud[1], stud[2], stud[3], stud[4], stud[5], stud[6], stud[7], stud[8]))
    stud_course_cassandra.append(stud)

session_postgres.close()

In [28]:
limpar_tabela('students_teaches')

query = """
SELECT
    i.id AS id_instructor,
    i.name,
    i.salary,
    s.id AS id_student,
    s.name,
    s.dept_name,
    s.tot_cred
FROM
    instructor i
JOIN
    student s on i.dept_name = s.dept_name;
"""
stud_teach = conn.execute(text(query)).all()

stud_teach_cassandra = []
		
for stud in stud_teach:
    query = f"INSERT INTO students_teaches(id_instructor, name, salary, id_student, name_student, dept_name, tot_cred) VALUES (%s, %s, %s, %s, %s, %s, %s);"
    session.execute(query, (stud[0], stud[1], stud[2], stud[3], stud[4], stud[5], stud[6]))
    stud_teach_cassandra.append(stud)

session_postgres.close()

In [29]:
limpar_tabela('student_advisor')

query = """
select
    i.id AS id_instructor,
    i.name AS name_instructor,
    s.id AS id_student,
	s.name AS name_student
from
	instructor i
join
	advisor a on i.id = a.i_id
join
	student s on a.s_id = s.id;
"""
student = conn.execute(text(query)).all()

student_advisor_cassandra = []

for stud in student:
    query = f"INSERT INTO student_advisor(id_instructor, name_instructor, id_student, name_student) VALUES (%s, %s, %s, %s);"
    session.execute(query, (stud[0], stud[1], stud[2], stud[3]))
    student_advisor_cassandra.append(stud)

session_postgres.close()

In [30]:
limpar_tabela('classroom_section')

query = """
select
	c.building,
	c.room_number,
	c.capacity
from
	classroom c
left join
	section s on c.building = s.building
where
	s.course_id is null;
"""
student = conn.execute(text(query)).all()

student_advisor_cassandra = []

for stud in student:
    query = f"INSERT INTO classroom_section(building, room_number, capacity) VALUES (%s, %s, %s);"
    session.execute(query, (stud[0], stud[1], stud[2]))
    student_advisor_cassandra.append(stud)

session_postgres.close()

In [31]:
limpar_tabela('instructor_student')

query = """
select 
    i.id as id_instructor,
    i.name,
    i.salary ,
    s.id as id_student,
    s.name,
    s.dept_name,
    s.tot_cred
from
    instructor i
join
    student s on i.dept_name = s.dept_name;
"""
student = conn.execute(text(query)).all()

instructor_student_cassandra = []

for stud in student:
    query = f"INSERT INTO instructor_student(id_instructor, name, salary, id_student, name_student, dept_name, tot_cred) VALUES (%s, %s, %s, %s, %s, %s, %s);"
    session.execute(query, (stud[0], stud[1], stud[2], stud[ 3], stud[4], stud[5], stud[6]))
    instructor_student_cassandra.append(stud)

session_postgres.close()

### Questões

#### Questão 1 - Listar todos os cursos oferecidos por um determinado departamento

In [32]:
def cursos_por_departamento(dept):
    cursos = session.execute(f"SELECT course_id, title, credits FROM course_department WHERE dept_name = '{dept}';")
    
    # Criar o DataFrame diretamente
    df = pd.DataFrame(cursos, columns=['course_id', 'title', 'credits'])
    
    return df

# Chamando a função e exibindo o DataFrame
df = cursos_por_departamento('Comp. Sci.')
df

,course_id,title,credits
0,CS-101,Intro. to Computer Science,4
1,CS-190,Game Design,4
2,CS-315,Robotics,3
3,CS-319,Image Processing,3
4,CS-347,Database System Concepts,3


#### Questão 2 - Recuperar todas as disciplinas de um curso específico em um determinado semestre

In [33]:
import pandas as pd

def disciplinas_por_curso_e_semestre(curso, semestre):
    # Consulta SQL para recuperar as disciplinas de um curso específico em um semestre específico
    query = f"""
    SELECT
        course_id,
        semester,
        title,
        year
    FROM
        section
    WHERE
        course_id = '{curso}' AND semester = '{semestre}';
    """
    
    disciplinas = session.execute(query)
    
    df = pd.DataFrame(disciplinas, columns=['course_id', 'title', 'semester', 'year'])
    
    return df

df = disciplinas_por_curso_e_semestre('CS-101', 'Fall')
df

,course_id,title,semester,year
0,CS-101,Fall,Intro. to Computer Science,2017


#### Questão 3 - Encontrar todos os estudantes que estão matriculados em um curso específico

In [34]:
def estudantes_por_curso(curso):
    query = f"""
    SELECT
        course_id,
        name,
        id,
        tot_cred
    FROM
        students_course
    WHERE
        course_id = '{curso}'
    ALLOW FILTERING;
    """
    
    estudantes = session.execute(query)
    
    df = pd.DataFrame(estudantes, columns=['course_id', 'name', 'id', 'tot_cred'])
    
    return df

df = estudantes_por_curso('CS-101')
df

,course_id,name,id,tot_cred
0,CS-101,Williams,54321,54
1,CS-101,Zhang,00128,102
2,CS-101,Brown,76543,58
3,CS-101,Shankar,12345,32


#### Questão 4 - Listar a média de salários de todos os professores em um determinado departamento

In [35]:
def media_salario_departamento(dept):
    d = 'Comp. Sci.'
    query = f"""
    SELECT
        dept_name,
        salary
    FROM
        students_teaches
    WHERE
        dept_name = '{d}';
    """

    resultado = session.execute(query)

    data = [{'dept_name': c.dept_name, 'salary': int(c.salary)} for c in resultado]

    df = pd.DataFrame(data)
    
    return df

df = media_salario_departamento('Comp. Sci.')
print(f"Média: {df['salary'].mean()}")
df

Média: 77333.33333333333


,dept_name,salary
0,Comp. Sci.,75000
1,Comp. Sci.,75000
2,Comp. Sci.,75000
3,Comp. Sci.,75000
4,Comp. Sci.,92000
5,Comp. Sci.,92000
6,Comp. Sci.,92000
7,Comp. Sci.,92000
8,Comp. Sci.,65000
9,Comp. Sci.,65000


#### Questão 5 - Recuperar o número total de créditos obtidos por um estudante específico

In [36]:
def total_creditos_por_estudante(nome_estudante):
    query = f"""
    SELECT name, credits
    FROM students_course
    WHERE name = '{nome_estudante}';
    """
    
    resultado = session.execute(query)
    
    dados_lista = [(row[0], int(row[1])) for row in resultado]
    
    df = pd.DataFrame(dados_lista, columns=['nome', 'creditos'])
    
    return df

df = total_creditos_por_estudante('Tanaka')
print(f'Total de creditos: {df["creditos"].sum()}')
df

Total de creditos: 11


,nome,creditos
0,Tanaka,3
1,Tanaka,4
2,Tanaka,4


#### Questão 6 - Encontrar todas as disciplinas ministradas por um professor em um semestre específico

In [37]:
def disciplinas_instrutor_semeste(instructor, semester):
    query = f"""
    SELECT
        id,
        name,
        course_id,
        semester,
        title
    FROM
        instructor_teaches
    WHERE
        name = '{instructor}'
        AND semester = '{semester}'
    ALLOW FILTERING;
    """
    
    instructor = session.execute(query)
    
    df = pd.DataFrame(instructor, columns=['id', 'name', 'course_id', 'semester', 'title'])
    
    return df

df = disciplinas_instrutor_semeste('Srinivasan', '1')
df

,id,name,course_id,semester,title


#### Questão 7 - Listar todos os estudantes que têm um determinado professor como orientador

In [38]:
def orientador_aluno(instructor):
    query = f"""
    SELECT
        name_instructor,
        name_student
    FROM
        student_advisor
    WHERE
        name_instructor = '{instructor}'
    ALLOW FILTERING;
    """
    
    students = session.execute(query)
    
    df = pd.DataFrame(students, columns=['name_student', 'name_student'])
    
    return df

df = orientador_aluno('Katz')
df

,name_student,name_student
0,Katz,Brown
1,Katz,Zhang


#### Questão 8 - Recuperar todas as salas de aula sem um curso associado

In [39]:
def sala_curso():
    query = f"""select * from classroom_section;"""
    
    students = session.execute(query)
    
    df = pd.DataFrame(students, columns=['building', 'capacity', 'room_number'])
    
    return df

df = sala_curso()
df

,building,capacity,room_number
0,Vazio,5,10


#### Questão 9 - Encontrar todos os pré-requisitos de um curso específico

#### Questão 10 - Recuperar a quantidade de alunos orientados por cada professor

In [40]:
def orientados_professor():
    query = f"""SELECT name, count(id_instructor) AS qtd  FROM instructor_student GROUP BY name;"""
    
    orientados = session.execute(query)
    
    df = pd.DataFrame(orientados, columns=['name', 'qtd'])
    
    return df

df = orientados_professor()
df.sort_values('qtd', ascending=False)

,name,qtd
3,Katz,4
5,Brandt,4
11,Srinivasan,4
6,Gold,3
9,Einstein,3
7,Kim,2
0,Mozart,1
1,Singh,1
2,El Said,1
4,Wu,1


In [41]:
# Listar todos os estudantes que têm um determinado professor como orientador

# listar_colunas('students_course')

# Row(name='Levy', title='Physical Principles', dept_name='Physics', budget='70000', building='Watson', course_id='PHY-101', credits='4', id='45678', tot_cred='46')
# Row(name='Sanchez', title='Music Video Production', dept_name='Music', budget='80000', building='Packard', course_id='MU-199', credits='3', id='55739', tot_cred='38')
# Row(name='Chavez', title='Investment Banking', dept_name='Finance', budget='120000', building='Painter', course_id='FIN-201', credits='3', id='23121', tot_cred='110')
# Row(name='Tanaka', title='Computational Biology', dept_name='Biology', budget='90000', building='Watson', course_id='BIO-399', credits='3', id='98988', tot_cred='120')
# Row(name='Tanaka', title='Genetics', dept_name='Biology', budget='90000', building='Watson', course_id='BIO-301', credits='4', id='98988', tot_cred='120')
# Row(name='Tanaka', title='Intro. to Biology', dept_name='Biology', budget='90000', building='Watson', course_id='BIO-101', credits='4', id='98988', tot_cred='120')

In [42]:
# listar_colunas('course_department')
# listar_colunas('section')
# listar_colunas('students_course')
# listar_colunas('students_teaches')
# listar_colunas('course')

In [43]:
# stud = session.execute("SELECT * FROM students_course;")
# for s in stud:
#     print(s)